In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data)
# data[:,0] # prints the first column (label)
m, n = data.shape
np.random.shuffle(data) # to prevent the model from learning unintended biases

# validation set
data_dev = data[0:1000].T # first 1000 examples, and transposing, so one column instead of row reprsnts an image now ( just for easiness to do calculations, if using tensorflow or pytorch no need to transpose, these libraries are written to handle rows )
Y_dev = data_dev[0] # so instead of [:, 0](for columns), use [0] now to return the label array (now the first row instead of first column)
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

# training set
data_train = data[1000:m].T # remaining examples from 1000 to m
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

# print(X_train[:, 0].shape) # should print 784 i.e. the number of items in each column which is the number of pixels per image
print(X_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
# initialize parameters
# creating a neural net with 3 layers
# input layer wiht 784 units
# 1st and 2nd (output) layer with 10 units
# all random at first, then adjusted later in backprop

def init_params():
    W1 = np.random.rand(10, 784) - 0.5 # weights for the layer (comin from input layer)
    b1 = np.zeros((10, 1)) # biases for the 1st layer
    W2 = np.random.rand(10, 10) - 0.5 # weights for the output layer
    b2 = np.zeros((10, 1)) # biases for the output layer
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z,0) # rectified linear unit, an activation function

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A # another activation func used at the output layer to get probabilities (betn 0 and 1)
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1  # Z1 is the unactivated 1st layer
    A1 = ReLU(Z1)        # Activated Z1 (A1)
    Z2 = W2.dot(A1) + b2 # same thing
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y): # remember Y is a row matrix, one_hot creates the matrix [0,0,1,0,...,0] for '2' and likewise for all images, this process is called one-hot encoding, look it up for more info
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    # Y.size is the number of examples, Y.max is 9 so +1 is 10, creates a zero matrix of size m x 10

    one_hot_Y[np.arange(Y.size), Y] = 1
    # a cool way of one hot encoding 

    one_hot_Y = one_hot_Y.T # each column is an example now
    return one_hot_Y
    
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): # alpha, the learning rate, is what we pass (custom)
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [10]:
def get_predictions(A2): # activated 2nd layer
    return np.argmax(A2, 0) # argmax returns the index of the highest value (probability in this case)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size
    # sum(predictions == Y) counts the number of True's we get from comparing predictions and Y
    # then we divide that by actual size of Y to get accuracy
    
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0: # every 10 iterations, we're going to check the progress
            print("Iteration: ", i)
            predictions = get_predictions(A2) 
            print(get_accuracy(predictions, Y)) 
    return W1, b1, W2, b2

In [11]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[1 4 1 ... 1 3 3] [8 1 9 ... 7 8 5]
0.072
Iteration:  10
[6 6 8 ... 7 6 6] [8 1 9 ... 7 8 5]
0.20560975609756096
Iteration:  20
[8 6 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.3110487804878049
Iteration:  30
[8 6 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.3999756097560976
Iteration:  40
[8 6 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.4690731707317073
Iteration:  50
[8 8 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.5227317073170732
Iteration:  60
[8 8 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.5678780487804878
Iteration:  70
[8 8 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.605609756097561
Iteration:  80
[8 8 9 ... 7 3 5] [8 1 9 ... 7 8 5]
0.6362439024390244
Iteration:  90
[8 8 9 ... 1 3 5] [8 1 9 ... 7 8 5]
0.6599268292682927
Iteration:  100
[8 8 9 ... 1 3 5] [8 1 9 ... 7 8 5]
0.680829268292683
Iteration:  110
[8 8 9 ... 1 3 5] [8 1 9 ... 7 8 5]
0.6979512195121951
Iteration:  120
[8 8 9 ... 1 3 5] [8 1 9 ... 7 8 5]
0.7112682926829268
Iteration:  130
[8 8 9 ... 1 3 5] [8 1 9 ... 7 8 5]
0.7235853658536585
Iteration:  140
[8 8 9 ... 1 3 